In [ ]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input
password = getpass.getpass()

# Note that when you use _SQLAlchemy_ and establish the connection, you do not even need to be logged in Sequel Pro or MySQL Workbench.

Establish a connection between Python and the Sakila database.

In [ ]:
bd = "sakila"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/'+bd
engine = create_engine(connection_string)
engine

Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
month: an integer representing the month for which rental data is to be retrieved.
year: an integer representing the year for which rental data is to be retrieved.
The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [ ]:
from sqlalchemy import text

with engine.connect() as connection:
    query = text('SELECT * FROM rental')
    result = connection.execute(query)

result

In [ ]:
def rentals_month(engine, month, year):
    with engine.connect() as connection:
        query = text(
            f'''SELECT *
            FROM rental
            WHERE YEAR(rental_date) = {year} AND MONTH(rental_date) = {month};
            ''')
        
        result = connection.execute(query)
        df = pd.DataFrame(result.all())
    return df

In [ ]:
rentals_5_2005 = rentals_month(engine, 5, 2005)
rentals_5_2005

Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()

In [ ]:
rentals_5_2005['customer_id'].nunique()

In [ ]:
def rental_count_month(df, month, year):
    result = df.groupby('customer_id')['rental_id'].count().reset_index()
    result.rename(columns={'rental_id': f'rental_{month}_{year}'}, inplace=True)
    return result

In [ ]:
rental_count_5_2005 = rental_count_month(rentals_5_2005, 5, 2005)

In [ ]:
rental_count_5_2005

Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [ ]:
rentals_6_2005 = rentals_month(engine, 6, 2005)
rentals_6_2005

In [ ]:
rental_count_6_2005 = rental_count_month(rentals_6_2005, 6, 2005)
rental_count_6_2005

In [ ]:
merged = pd.merge(rental_count_5_2005, rental_count_6_2005, on=['customer_id'])
merged['difference'] = merged['rental_5_2005'] - merged['rental_6_2005']
merged

In [ ]:
def compare_rentals(df1, df2):
    merged = pd.merge(df1, df2, on=['customer_id'])
    merged['difference'] = merged.iloc[:, 1] - merged.iloc[:, 2]
    return merged

In [ ]:
merged_df = compare_rentals(rental_count_5_2005, rental_count_6_2005)
merged_df